In [1]:
## Find keas by name
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import pandas as pd
import requests
import json

## API Settings

In [6]:
url = "https://data.keadatabase.nz/band_combos/?style=&study_area=&bird__status=alive&colours=&symbols=&is_extended=true&is_featured=unknown"

## Fetch keas

In [7]:
def get_keas(url = ""):
    response = requests.get(url).json()
    
    if response['results']:
        df = pd.io.json.json_normalize(response['results'])
        df.columns = df.columns.map(lambda x: x.split(".")[-1])
 
    else:
        print('no data')
    if response['next']:
        df = pd.concat([df,  get_keas(response['next'])], ignore_index=True,sort=False)
         
    return df

In [8]:
keas = get_keas(url)

## Explore data & Drop duplicates

In [9]:
print(keas.shape)
print(keas.columns)
print(keas.dtypes)
keas = keas.loc[:,~keas.columns.duplicated()]

(228, 29)
Index(['band_combo', 'description', 'is_featured', 'full_size', 'large',
       'thumbnail', 'profile_picture_attribution', 'sponsor_name',
       'sponsor_website', 'birthday', 'date_imported', 'date_modified',
       'get_age', 'get_life_stage', 'name', 'primary_band', 'sex', 'slug',
       'status', 'study_area', 'colours', 'date_deployed', 'date_imported',
       'date_modified', 'name', 'special', 'study_area', 'style', 'symbols'],
      dtype='object')
band_combo                     object
description                    object
is_featured                      bool
full_size                      object
large                          object
thumbnail                      object
profile_picture_attribution    object
sponsor_name                   object
sponsor_website                object
birthday                       object
date_imported                  object
date_modified                  object
get_age                         int64
get_life_stage                 ob

In [16]:
keas.head()

,band_combo,description,is_featured,full_size,large,thumbnail,profile_picture_attribution,sponsor_name,sponsor_website,birthday,...,primary_band,sex,slug,status,study_area,colours,date_deployed,special,style,symbols
0,Red o on White,Addi was one of ten kea banded at Moturau hut ...,False,NaN,NaN,NaN,None,None,None,2019-01-01,...,V-3209,Female,addi,Alive,Fiordland,"[red, white]",2019-08-20,●,new,[●]
1,White C on Orange,Two kea were caught together at Red Tarns on t...,False,https://media.keadatabase.nz/media/birds/adria...,https://media.keadatabase.nz/media/__sized__/b...,https://media.keadatabase.nz/media/__sized__/b...,Clio Reid,None,None,2019-01-01,...,V-2103,Female,adriana,Alive,Aoraki,"[white, orange]",2019-03-14,None,new,[C]
2,Yellow * on Black,Alwyn hangs out mostly at Deaths Corner and Ca...,False,https://media.keadatabase.nz/media/birds/alwyn...,https://media.keadatabase.nz/media/__sized__/b...,https://media.keadatabase.nz/media/__sized__/b...,None,None,None,2014-01-01,...,V-2626,Female,alwyn,Alive,Waimakariri,"[yellow, black]",2016-08-09,None,new,[*]
3,x / x - Metal / x,Angelina has a thick steel band. Angelina was ...,False,https://media.keadatabase.nz/media/birds/angel...,https://media.keadatabase.nz/media/__sized__/b...,https://media.keadatabase.nz/media/__sized__/b...,Willowbank,None,None,2006-10-22,...,None,Female,angelina,Alive,Willowbank,[metal],2017-10-13,None,old,[]
4,White S on Red,Anthea is one of the earlier birds banded with...,False,https://media.keadatabase.nz/media/birds/anthe...,https://media.keadatabase.nz/media/__sized__/b...,https://media.keadatabase.nz/media/__sized__/b...,Laura Young,None,None,2009-01-01,...,V-1299,Female,anthea,Alive,Waimakariri,"[white, red]",2013-09-13,None,new,[S]


In [15]:
keas['study_area'].value_counts()

Waimakariri    142
Fiordland       34
Willowbank      12
Wanaka           9
Westland         8
Orana Park       7
Aoraki           6
Rotoiti          5
Kahurangi        3
Wakatipu         1
Mt Richmond      1
Name: study_area, dtype: int64

## Remove birds with no thumbnail

In [14]:
import numpy as np
kea_df = keas.dropna(subset=['thumbnail'])
kea_df.to_csv('keas-with-images.csv')
kea_df.sex.value_counts()

Male            150
Female           68
Undetermined      6
Name: sex, dtype: int64